Done using a Sharded Model

In [ ]:
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

def load_quantized_model(model_name: str):
  bnb_config = BitsAndBytesConfig(
      load_in_4bit = True,    # Weights and activation represented using4 bit instead of 32
      bnb_4bit_use_double_quant = True,   # Does another quantization
      bnb_4bit_quant_type = "nf4",     # default quant type
      bnb_4bit_compute_dtype= torch.bfloat16
  )

  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      torch_dtype = torch.bfloat16,
      quantization_config = bnb_config
  )

  return model

def initialize_tokenizer(model_name: str):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.bos_token_id = 1
  return tokenizer

model = load_quantized_model(model_name)
tokenizer = initialize_tokenizer(model_name)
stop_token_ids = [0]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
text = "[INST] Provide me a step by step approach to become a Deep Learning Research Scientist. [/INST]"
# INST Required since it is a instruction based model

# Generating torch tensors as outputs
encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)

model_input = encoded
generated_ids = model.generate(**model_input, max_new_tokens=500, do_sample=True)

# decoding the tensors
decoded = tokenizer.batch_decode(generated_ids)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1506: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
print(decoded[0])

[INST] How AI will replace Engineers [/INST] AI is not likely to Question: "How will AI replace engineers?" as this is a complex question that requires a deep understanding of artificial intelligence, engineering, and their interaction.

AI can augment and assist engineers in various ways: from design and simulation to prediction and optimization, but it cannot completely replace them. Engineering is a profession that requires a high level of creativity, inventiveness, critical thinking, and problem-solving skills. These are areas where AI currently falls short, and where human engineers can excel.

Moreover, the role of engineers is constantly evolving; new technologies and challenges require engineers to adapt, acquire new skills, and find innovative solutions. AI can be used as a tool to help engineers in their work, but it cannot replace them entirely.

However, AI can be used to automate repetitive and standardized tasks, allowing engineers to focus on more complex problems and cr